# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [5]:
!pip install langchain_experimental
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00


In [6]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [7]:
!pip install faiss-cpu > /dev/null
!pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [9]:
from google.colab import userdata
OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
SERPAPI_API_KEY = userdata.get('SERPAPI_KEY')


In [11]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings(api_key= OPENAI_API_KEY)
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [16]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0, api_key = OPENAI_API_KEY), prompt=todo_prompt)
search = SerpAPIWrapper(serpapi_api_key = SERPAPI_API_KEY)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [18]:
llm = OpenAI(api_key= OPENAI_API_KEY,temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

<ipython-input-18-69e1cc6be7ef>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [24]:
OBJECTIVE = "visit Scotland in April"

In [25]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [26]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to create a list of tasks to complete for a specific objective
Action: TODO
Action Input: visit Scotland in April
Observation: 

1. Research flights and accommodations: Start by researching flights and accommodations for your trip to Scotland in April. Look for deals and compare prices to find the best option for your budget.

2. Decide on a budget: Set a budget for your trip and make sure to stick to it. Consider the cost of flights, accommodations, transportation, food, and activities.

3. Create an itinerary: Plan out your daily activities and sightseeing for each day of your trip. Consider the weather and any special events or festivals happening in Scotland during April.

4. Book flights and accommodations: Once you have decided on your itinerary and budget, book your flights and accommodations. Make sure to read reviews and choose reputable o

ValueError: Tried to add ids that already exist: {'result_1_0'}